In [ ]:
import pandas as pd
import numpy as np

# import matplotlib as plt
import matplotlib.pyplot as plt
import scipy
from skspatial.objects import LineSegment, Line, Vector
from enum import Enum

Layer = Enum("Layer", "FRONT BACK")

In [ ]:
VIA_DIAM = 0.8
VIA_DRILL = 0.4
STATOR_HOLE_RADIUS = 5
TRACK_WIDTH = 0.127
TRACK_SPACING = 0.127
TURNS = 18
STATOR_RADIUS = 18
COIL_CENTER_RADIUS = 11.5
# where to place the pins
CONNECTION_PINS_RADIUS = 16
USE_SPIRAL = False

In [ ]:
# get the point on an arc at the given angle
def get_arc_point(angle, radius):
    return (
        radius * np.cos(np.deg2rad(angle)),
        radius * np.sin(np.deg2rad(angle)),
    )


# draw an arc
def draw_arc(start_angle, end_angle, radius, step=10):
    points = []
    for angle in np.arange(start_angle, end_angle + step, step):
        x = radius * np.cos(np.deg2rad(angle))
        y = radius * np.sin(np.deg2rad(angle))
        points.append((x, y))
    return points


# roate the points by the required angle
def rotate(points, angle):
    return [
        [
            x * np.cos(np.deg2rad(angle)) - y * np.sin(np.deg2rad(angle)),
            x * np.sin(np.deg2rad(angle)) + y * np.cos(np.deg2rad(angle)),
        ]
        for x, y in points
    ]


# move the points out to the distance at the requited angle
def translate(points, distance, angle):
    return [
        [
            x + distance * np.cos(np.deg2rad(angle)),
            y + distance * np.sin(np.deg2rad(angle)),
        ]
        for x, y in points
    ]


# flip the y coordinate
def flip_y(points):
    return [[x, -y] for x, y in points]


def flip_x(points):
    return [[-x, y] for x, y in points]

# Arbitrary Coil Generation

In [ ]:
template_f = [
    (-0.6, 0),
    (-0.6, -0.6),
    (0.5, -1.2),
    (0.95, -0.4),
    (0.95, 0),
    (0.95, 0.4),
    (0.5, 1.2),
    (-0.6, 0.6),
]

template_b = []
for i in range(len(template_f)):
    template_b.append(template_f[len(template_f) - i - len(template_f) // 2])
template_b = flip_x(template_b)

In [ ]:
# plot the template shape wrapping around to the first point
df = pd.DataFrame(template_f + [template_f[0]], columns=["x", "y"])
ax = df.plot.line(x="x", y="y", color="blue")
ax.axis("equal")
df = pd.DataFrame(template_b + [template_b[0]], columns=["x", "y"])
ax = df.plot.line(x="x", y="y", color="red", ax=ax)

In [ ]:
def calculate_point(point, point1, point2, spacing, turn, layer):
    reference_vector = Vector([-100, 0])
    angle = np.rad2deg(Vector(point).angle_between(reference_vector))
    if point[1] > 0:
        angle = 360 - angle
    vector = Vector(point1) - Vector(point2)
    normal = vector / np.linalg.norm(vector)
    # rotate the vector 90 degrees
    normal = np.array([-normal[1], normal[0]])
    # move the  point along the normal vector by the spacing
    offset = spacing * (turn * 360 + angle) / 360
    coil_point = point + normal * offset
    return (coil_point[0], coil_point[1])


def get_points(template, turns, spacing, layer=Layer.FRONT):
    coil_points = []
    reference_vector = Vector([-100, 0])
    template_index = 0
    template_length = len(template)
    for turn in range(turns * template_length):
        point1 = template[template_index % template_length]
        point2 = template[(template_index + 1) % template_length]

        # calculate the new positions of the points
        coil_point1 = calculate_point(
            point1, point1, point2, spacing, template_index // template_length, layer
        )
        coil_point2 = calculate_point(
            point2,
            point1,
            point2,
            spacing,
            (template_index + 1) // template_length,
            layer,
        )
        # add an intermediate point which is the intersection of this new line with the previous line (if there is one)
        if len(coil_points) >= 2:
            # create a line from the previous two points
            line1 = Line(
                coil_points[len(coil_points) - 2],
                np.array(coil_points[len(coil_points) - 1])
                - np.array(coil_points[len(coil_points) - 2]),
            )
            # create a line from the two new points
            line2 = Line(
                np.array(coil_point1),
                np.array(np.array(coil_point1) - np.array(coil_point2)),
            )
            # find the intersection of the two lines
            try:
                intersection = line1.intersect_line(line2)
                coil_points.append(intersection)
            except:
                pass
        coil_points.append(coil_point1)
        coil_points.append(coil_point2)

        template_index = template_index + 1
    return coil_points


def optimize_points(points):
    # follow the line and remove points that are in the same direction as the previous poin
    # keep doing this until the direction changes significantly
    # this is a very simple optimization that removes a lot of points
    # it's not perfect but it's a good start
    optimized_points = []
    for i in range(len(points)):
        if i == 0:
            optimized_points.append(points[i])
        else:
            vector1 = np.array(points[i]) - np.array(points[i - 1])
            vector2 = np.array(points[(i + 1) % len(points)]) - np.array(points[i])
            length1 = np.linalg.norm(vector1)
            length2 = np.linalg.norm(vector2)
            if length1 > 0 and length2 > 0:
                dot = np.dot(vector1, vector2) / (length1 * length2)
                # clamp dot between -1 and 1
                dot = max(-1, min(1, dot))
                angle = np.arccos(dot)
                if angle > np.deg2rad(5):
                    optimized_points.append(points[i])
    print("Optimised from {} to {} points".format(len(points), len(optimized_points)))
    return optimized_points


def chaikin(points, iterations):
    if iterations == 0:
        return points
    l = len(points)
    smoothed = []
    for i in range(l - 1):
        x1, y1 = points[i]
        x2, y2 = points[i + 1]
        smoothed.append([0.9 * x1 + 0.1 * x2, 0.9 * y1 + 0.1 * y2])
        smoothed.append([0.1 * x1 + 0.9 * x2, 0.1 * y1 + 0.9 * y2])
    smoothed.append(points[l - 1])
    return chaikin(smoothed, iterations - 1)

In [ ]:
if not USE_SPIRAL:
    points_f = chaikin(
        optimize_points(
            flip_x(
                get_points(template_b, TURNS, TRACK_SPACING + TRACK_WIDTH, Layer.FRONT)
            )
        ),
        2,
    )
    points_b = chaikin(
        optimize_points(
            get_points(template_f, TURNS, TRACK_SPACING + TRACK_WIDTH, Layer.BACK)
        ),
        2,
    )

    points_f = [(0, 0)] + points_f
    points_b = [(0, 0)] + points_b

    df = pd.DataFrame(points_f, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="blue")
    ax.axis("equal")
    df = pd.DataFrame(points_b, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="red", ax=ax)

    print("Track points", len(points_f), len(points_b))
else:
    print("Using spiral")

# Basic Spiral Coil Generation

In [ ]:
def get_spiral(turns, start_radius, thickness, layer=Layer.FRONT):
    points = []
    # create a starting point in the center
    for angle in np.arange(0, turns * 360, 1):
        radius = start_radius + thickness * angle / 360
        if layer == Layer.BACK:
            x = radius * np.cos(np.deg2rad(angle + 180))
            y = radius * np.sin(np.deg2rad(angle + 180))
            points.append((x, -y))
        else:
            x = radius * np.cos(np.deg2rad(angle))
            y = radius * np.sin(np.deg2rad(angle))
            points.append((x, y))
    return points

In [ ]:
if USE_SPIRAL:
    points_f = get_spiral(
        TURNS, VIA_DIAM / 2 + TRACK_SPACING, TRACK_SPACING + TRACK_WIDTH, Layer.FRONT
    )
    points_b = get_spiral(
        TURNS, VIA_DIAM / 2 + TRACK_SPACING, TRACK_SPACING + TRACK_WIDTH, Layer.BACK
    )

    points_f = [(0, 0)] + points_f
    points_b = [(0, 0)] + points_b
    print("Track points", len(points_f), len(points_b))
else:
    print("Using template")

# Generate PCB Layout

In [ ]:
# calculat the total length of the track to compute the resistance
total_length_front = 0
for i in range(len(points_f) - 1):
    total_length_front += np.linalg.norm(
        np.array(points_f[i + 1]) - np.array(points_f[i])
    )
print("Total length front", total_length_front)

total_length_back = 0
for i in range(len(points_b) - 1):
    total_length_back += np.linalg.norm(
        np.array(points_b[i + 1]) - np.array(points_b[i])
    )
print("Total length back", total_length_back)

In [ ]:
vias = []
tracks_f = []
tracks_b = []
pads = []

angle_A = 0
angle_B = 120
angle_C = 240


def create_pad(radius, angle, name):
    return {
        "x": radius * np.cos(np.deg2rad(angle)),
        "y": radius * np.sin(np.deg2rad(angle)),
        "name": name,
    }


# create the pads at CONNECTION_PINS radius - 2 for each of the coils, A, B and C
pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_A - 30, "A"))
pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_A + 30, "A"))

pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_B - 30, "B"))
pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_B + 30, "B"))

pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_C - 30, "C"))
pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_C + 30, "C"))


def create_via(point):
    return {"x": point[0], "y": point[1]}


# the main coils
coil_A_f = translate(rotate(points_f, angle_A), COIL_CENTER_RADIUS, angle_A)
coil_A_b = translate(rotate(points_b, angle_A), COIL_CENTER_RADIUS, angle_A)
tracks_f.append(coil_A_f)
tracks_b.append(coil_A_b)

coil_B_f = translate(rotate(points_f, angle_B), COIL_CENTER_RADIUS, angle_B)
coil_B_b = translate(rotate(points_b, angle_B), COIL_CENTER_RADIUS, angle_B)
tracks_f.append(coil_B_f)
tracks_b.append(coil_B_b)

coil_C_f = translate(rotate(points_f, angle_C), COIL_CENTER_RADIUS, angle_C)
coil_C_b = translate(rotate(points_b, angle_C), COIL_CENTER_RADIUS, angle_C)
tracks_f.append(coil_C_f)
tracks_b.append(coil_C_b)

# the opposite coils - for more power!
angle_A_opp = angle_A + 180
angle_B_opp = angle_B + 180
angle_C_opp = angle_C + 180

coil_A_opp_f = translate(
    rotate(flip_y(points_f), angle_A_opp), COIL_CENTER_RADIUS, angle_A_opp
)
coil_A_opp_b = translate(
    rotate(flip_y(points_b), angle_A_opp), COIL_CENTER_RADIUS, angle_A_opp
)
tracks_f.append(coil_A_opp_f)
tracks_b.append(coil_A_opp_b)

coil_B_opp_f = translate(
    rotate(flip_y(points_f), angle_B_opp), COIL_CENTER_RADIUS, angle_B_opp
)
coil_B_opp_b = translate(
    rotate(flip_y(points_b), angle_B_opp), COIL_CENTER_RADIUS, angle_B_opp
)
tracks_f.append(coil_B_opp_f)
tracks_b.append(coil_B_opp_b)

coil_C_opp_f = translate(
    rotate(flip_y(points_f), angle_C_opp), COIL_CENTER_RADIUS, angle_C_opp
)
coil_C_opp_b = translate(
    rotate(flip_y(points_b), angle_C_opp), COIL_CENTER_RADIUS, angle_C_opp
)
tracks_f.append(coil_C_opp_f)
tracks_b.append(coil_C_opp_b)

# connect the front and back coils together
vias.append(create_via(get_arc_point(angle_A, COIL_CENTER_RADIUS)))
vias.append(create_via(get_arc_point(angle_B, COIL_CENTER_RADIUS)))
vias.append(create_via(get_arc_point(angle_C, COIL_CENTER_RADIUS)))
vias.append(create_via(get_arc_point(angle_A_opp, COIL_CENTER_RADIUS)))
vias.append(create_via(get_arc_point(angle_B_opp, COIL_CENTER_RADIUS)))
vias.append(create_via(get_arc_point(angle_C_opp, COIL_CENTER_RADIUS)))

# connect the front copper opposite coils together
common_connection_radius = STATOR_RADIUS - (VIA_DIAM / 2 + TRACK_SPACING)
common_coil_connections_b = draw_arc(angle_A_opp, angle_C_opp, common_connection_radius)
coil_A_opp_f.append(get_arc_point(angle_A_opp, common_connection_radius))
coil_B_opp_f.append(get_arc_point(angle_B_opp, common_connection_radius))
coil_C_opp_f.append(get_arc_point(angle_C_opp, common_connection_radius))

tracks_b.append(common_coil_connections_b)

vias.append(create_via(get_arc_point(angle_A_opp, common_connection_radius)))
vias.append(create_via(get_arc_point(angle_B_opp, common_connection_radius)))
vias.append(create_via(get_arc_point(angle_C_opp, common_connection_radius)))

# connect the coils to the pads
coil_A_f.append(get_arc_point(angle_A, common_connection_radius))
coil_B_f.append(get_arc_point(angle_B, common_connection_radius))
coil_C_f.append(get_arc_point(angle_C, common_connection_radius))

tracks_f.append(
    [get_arc_point(angle_A - 30, CONNECTION_PINS_RADIUS)]
    + draw_arc(angle_A - 30, angle_A + 30, common_connection_radius)
    + [get_arc_point(angle_A + 30, CONNECTION_PINS_RADIUS)]
)
tracks_f.append(
    [get_arc_point(angle_B - 30, CONNECTION_PINS_RADIUS)]
    + draw_arc(angle_B - 30, angle_B + 30, common_connection_radius)
    + [get_arc_point(angle_B + 30, CONNECTION_PINS_RADIUS)]
)
tracks_f.append(
    [get_arc_point(angle_C - 30, CONNECTION_PINS_RADIUS)]
    + draw_arc(angle_C - 30, angle_C + 30, common_connection_radius)
    + [get_arc_point(angle_C + 30, CONNECTION_PINS_RADIUS)]
)

# wires for connecting to opposite coils
connection_radius1 = STATOR_HOLE_RADIUS + (TRACK_SPACING)
connection_radius2 = connection_radius1 + (TRACK_SPACING + VIA_DIAM / 2)

# draw a 45 degree line from each coil at connection radius 1
# then connect up to connection radius 2
# draw a 45 degree line to the opposite coil

# coil A
coil_A_b.append(get_arc_point(angle_A, connection_radius1))
coil_A_opp_b.append(get_arc_point(angle_A_opp, connection_radius2))
a_connection_b = draw_arc(angle_A, angle_A + 90, connection_radius1)
a_connection_f = draw_arc(angle_A + 90, angle_A + 180, connection_radius2)
a_connection_b.append(a_connection_f[0])

tracks_f.append(a_connection_f)
tracks_b.append(a_connection_b)

# coil B
coil_B_b.append(get_arc_point(angle_B, connection_radius1))
coil_B_opp_b.append(get_arc_point(angle_B_opp, connection_radius2))
b_connection_b = draw_arc(angle_B, angle_B + 90, connection_radius1)
b_connection_f = draw_arc(angle_B + 90, angle_B + 180, connection_radius2)
b_connection_b.append(b_connection_f[0])

tracks_f.append(b_connection_f)
tracks_b.append(b_connection_b)

# coil C
coil_C_b.append(get_arc_point(angle_C, connection_radius1))
coil_C_opp_b.append(get_arc_point(angle_C_opp, connection_radius2))
c_connection_b = draw_arc(angle_C, angle_C + 90, connection_radius1)
c_connection_f = draw_arc(angle_C + 90, angle_C + 180, connection_radius2)
c_connection_b.append(c_connection_f[0])

tracks_f.append(c_connection_f)
tracks_b.append(c_connection_b)

vias.append(create_via(a_connection_f[0]))
vias.append(create_via(b_connection_f[0]))
vias.append(create_via(c_connection_f[0]))

vias.append(create_via(a_connection_f[-1]))
vias.append(create_via(b_connection_f[-1]))
vias.append(create_via(c_connection_f[-1]))

In [ ]:
def create_track(points):
    return [{"x": x, "y": y} for x, y in points]


# dump out the results to json
json_result = {
    "parameters": {
        "trackWidth": TRACK_WIDTH,
        "statorHoleRadius": STATOR_HOLE_RADIUS,
        "statorRadius": STATOR_RADIUS,
        "viaDiameter": VIA_DIAM,
        "viaDrillDiameter": VIA_DRILL,
    },
    "vias": vias,
    "pads": pads,
    "silk": [
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_A)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_A)),
            "text": "A",
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_B)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_B)),
            "text": "B",
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_C)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_C)),
            "text": "C",
        },
    ],
    "tracks": {
        "f": [create_track(points) for points in tracks_f],
        "b": [create_track(points) for points in tracks_b],
    },
}

import json

json.dump(json_result, open("coil.json", "w"))


# df = pd.DataFrame(coil_A_f, columns=["x", "y"])
# ax = df.plot.line(x="x", y="y", label="Coil A", color="blue")
# ax.axis("equal")
# df = pd.DataFrame(coil_A_b, columns=["x", "y"])
# ax = df.plot.line(x="x", y="y", label="Coil B", color="green")
# ax.axis("equal")


# plot the back tracks
ax = None
for track in json_result["tracks"]["b"]:
    df = pd.DataFrame(track, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="blue", ax=ax)
    ax.axis("equal")

# plot the front tracks
for track in json_result["tracks"]["f"]:
    df = pd.DataFrame(track, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="red", ax=ax)
    ax.axis("equal")

# hide the legend
ax.legend().set_visible(False)
# make the plot bigger
ax.figure.set_size_inches(10, 10)

# plot the vias
for via in json_result["vias"]:
    ax.add_patch(
        plt.Circle(
            (via["x"], via["y"]),
            radius=VIA_DIAM / 2,
            fill=True,
            color="black",
        )
    )
    ax.add_patch(
        plt.Circle(
            (via["x"], via["y"]),
            radius=VIA_DRILL / 2,
            fill=True,
            color="white",
        )
    )

# plot the edge cuts
ax.add_patch(
    plt.Circle(
        (0, 0),
        radius=STATOR_RADIUS,
        fill=False,
        color="yellow",
    )
)
ax.add_patch(
    plt.Circle(
        (0, 0),
        radius=STATOR_HOLE_RADIUS,
        fill=False,
        color="yellow",
    )
)

# plot the pads
for pad in json_result["pads"]:
    ax.add_patch(
        plt.Circle(
            (pad["x"], pad["y"]),
            radius=1.7 / 2,
            fill=True,
            color="yellow",
        )
    )
    ax.add_patch(
        plt.Circle(
            (pad["x"], pad["y"]),
            radius=1.0 / 2,
            fill=True,
            color="white",
        )
    )